# Urban Air Pollution Challenge


In [ ]:
# Import of relevant packages
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

## Upload the data

In [ ]:
import sys
print(sys.executable)

In [ ]:
# Upload Train data
df_train=pd.read_csv('data/Train.csv')

# Upload Test data
df_test=pd.read_csv('data/Test.csv')

In [ ]:
df_train.columns

In [ ]:
# Choosing to drop the Place_ID X Date, as it doesn't contain any additional information
df_train = df_train.drop('Place_ID X Date', axis = 1)

In [ ]:
df_train.head(10)

## Split into train and validation set


In [ ]:
# separate columns in target values, 'id' and numerical features
target_vars = ['target', 'target_min', 'target_max', 'target_variance', 'target_count']
id_cols = ['Place_ID', 'Date']
num_cols = [col for col in df_train.columns if col not in target_vars + id_cols and pd.api.types.is_numeric_dtype(df_train[col])]

In [ ]:
X = df_train.drop(target_vars, axis=1)
Y = df_train['target']

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.20, random_state=42)

In [ ]:
X_train.info()

## Preliminary Data Cleaning

In [ ]:
# Check the number of nans for each column
missing = pd.DataFrame(X_train.isnull().sum(), columns=["Amount"])
missing['Percentage'] = round((missing['Amount']/X_train.shape[0])*100, 2)
missing[missing['Amount'] != 0]

In [ ]:
# Create missing data heatmap
plt.figure(figsize=(15, 8))

missing_data = X_train.isnull()
sns.heatmap(missing_data, yticklabels=False, cbar=True, cmap='viridis')
plt.title('Heatmap for check of missing data\n(Yellow = Missing, Dark = Present)', fontsize=14)
plt.xlabel('Features')
plt.ylabel('Observations')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

We remove all columns with more than 50% of missing data:

In [ ]:
cols = X_train.columns[X_train.isna().mean() > 0.5].tolist()

In [ ]:
X_train = X_train.drop(columns = cols, axis = 1)
X_train = X_train.reset_index(drop=True)

In [ ]:
X_val = X_val.drop(columns = cols, axis = 1)
X_val = X_val.reset_index(drop=True)

In [ ]:
X_train.info()

In [ ]:
X_train.describe()

## EDA


### Correlation between features

We here check the correlation between all features, as it appears the some have a high correlation with other features (corr >0.8). If that is the case, we drop one of the features

In [ ]:
import plotly.graph_objects as go


# Calculates de correlations matrix
corr_matrix = X_train.drop(['Place_ID','Date'], axis=1).corr()

# Create a mask
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
corr_masked = corr_matrix.where(~mask)

# Creates heatmap
fig = go.Figure(data=go.Heatmap(
    z=corr_masked.values,
    x=corr_masked.columns,
    y=corr_masked.columns,
    colorscale='RdBu_r',
    zmid=0,
    zmin=-1,
    zmax=1,
    text=np.round(corr_masked.values, 3),
    texttemplate='%{text}',
    textfont={"size": 8},
    colorbar=dict(title="Correlation"),
    hovertemplate='%{y} vs %{x}<br>Correlation: %{z:.3f}<extra></extra>'
))

fig.update_layout(
    title='Interactive Correlation Matrix of Features',
    title_font_size=16,
    width=1000,
    height=900,
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis={'side': 'bottom'},
    yaxis={'autorange': 'reversed'}
)

fig.update_xaxes(tickangle=45)
fig.update_layout(
    width=1400,   
    height=1200   
)
fig.show(renderer='browser')

In [ ]:
# Find features couples with corr > 0.8
threshold = 0.8
corr_matrix = X_train.drop(['Place_ID','Date'], axis=1).corr()

# Consider only upper triangle to avoid duplicates)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# find features that have corr > threshold
high_corr_indices = np.where((np.abs(corr_matrix) > threshold) & mask)

high_corr_pairs = [
    (corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j])
    for i, j in zip(*high_corr_indices)
]

high_corr_df = pd.DataFrame(high_corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])
high_corr_df = high_corr_df.sort_values('Correlation', ascending=False, key=abs)
high_corr_df = high_corr_df.reset_index(drop=True) 

print(f"Found {len(high_corr_df)} pairs with |correlation| > {threshold}")
print(high_corr_df)

In [ ]:
# Track which features have been dropped
features_to_drop = set()

# Iterate through each pair in high_corr_df
for idx, row in high_corr_df.iterrows():
    feature1 = row['Feature 1']
    feature2 = row['Feature 2']
    
    # If feature1 is already dropped, skip this pair
    if feature1 in features_to_drop:
        continue
    
    # If feature2 is already dropped, skip this pair
    if feature2 in features_to_drop:
        continue
    
    # Drop feature2 (the second feature in the pair)
    features_to_drop.add(feature2)
    print(f"Dropping {feature2} (correlated with {feature1}: {row['Correlation']:.3f})")

print(f"\n\nTotal features to drop: {len(features_to_drop)}")
print(f"Features: {sorted(features_to_drop)}")

# Drop from X_train and X_val
X_train = X_train.drop(columns=features_to_drop, axis=1)
X_val = X_val.drop(columns=features_to_drop, axis=1)

print(f"\nNew shapes after dropping correlated features:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")


In [ ]:
# df_eda = pd.concat([X_train, y_train], axis=1)

In [ ]:
# sns.pairplot(df_eda,corner=True);

### Correlations between features and target (PM2.5)


In [ ]:
X_num = X_train.drop(['Place_ID','Date'],axis = 1)

In [ ]:
correlations = pd.DataFrame({
    'feature': X_num.columns,
    'correlation': [X_num[col].corr(y_train.reset_index(drop=True)) for col in X_num.columns]
}).sort_values('correlation', ascending=False, key=abs)

print("Correlation of each feature with target:")
print(correlations)

In [ ]:
plt.figure(figsize=(10, 12))
colors = ['red' if x < 0 else 'green' for x in correlations['correlation']]
plt.barh(correlations['feature'], correlations['correlation'], color=colors, alpha=0.7)
plt.xlabel('Correlation with Target (PM2.5)', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('Feature Correlations with Target', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### Some other plots

## Feature engineering

### Preprocessing

In this part, we build the steps to input missing values and apply a standard scaling on the train set and the validation set.

For imputing, we consider the numerical columns:

In [ ]:
num_cols = [col for col in X_train.columns if col not in id_cols and pd.api.types.is_numeric_dtype(X_train[col])]

In [ ]:
X_train.columns

We impute the missing values using the mean per Place_ID

In [ ]:
# Fill missing values per Place_ID using the group mean
X_imputed = X_train.copy()
# X_imputed[num_cols] = X_train.groupby('Place_ID')[num_cols].transform(lambda x: x.fillna(x.mean()))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class GroupByPlaceIDImputer(BaseEstimator, TransformerMixin):
    def __init__(self, place_id='Place_ID', strategy='mean'):
        self.place_id = place_id
        self.strategy = strategy
        
    def fit(self, X, y=None):        
        self.group_means_ = X.groupby(self.place_id)[num_cols].mean()
        self.overall_means_ = X[num_cols].mean()
        self.num_cols_ = num_cols
        return self
    
    def transform(self, X):
        X_filled = X.copy()
        
        # Map Place_ID to group means
        for col in self.num_cols_:
            # Create a mapping: Place_ID -> mean value for this column
            place_to_mean = self.group_means_[col].to_dict()
            
            # Fill NaN values
            mask = X_filled[col].isna()
            X_filled.loc[mask, col] = X_filled.loc[mask, self.place_id].map(place_to_mean)
            
            # Fill remaining NaN (for Place_IDs not in training) with overall mean
            X_filled[col] = X_filled[col].fillna(self.overall_means_[col])
        
        return X_filled

# Transforemer
imputer = GroupByPlaceIDImputer(place_id='Place_ID', strategy='mean')
X_imputed = imputer.fit_transform(X_train)  # Transform all the Dataseet
X_imputed.info()

Inpute the missing values on the validation set:

In [ ]:
X_val_imputed = imputer.transform(X_val)

Check the missing values again, as there could still be some extra missing values:

In [ ]:
missing = pd.DataFrame(X_imputed.isnull().sum(), columns=["Amount"])
missing['Percentage'] = round((missing['Amount']/X_imputed.shape[0])*100, 2)
missing[missing['Amount'] != 0]

We now define a preprocessing pipeline to impute all the missing NaNs and to scale all the data with a standard scaler

In [ ]:
pipeline = Pipeline([
    # ('imputer', SimpleImputer(strategy='mean')),  # its optional to keep it as we already filled the missing values but its a safety layer for the future unseen data
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num', pipeline, num_cols),
], remainder='passthrough')

Column transformer changes the order of the columns, have to take this into account

In [ ]:
column_order = num_cols + ['Date','Place_ID']
X_preprocessed = pd.DataFrame(preprocessor.fit_transform(X_imputed), columns=column_order)

for col in num_cols:
    X_preprocessed[col] = pd.to_numeric(X_preprocessed[col], errors='coerce')


In [ ]:
X_preprocessed.info()

Apply the preprocessing pipeline on the validation set:

In [ ]:
X_val_preprocessed = pd.DataFrame(
    preprocessor.transform(X_val_imputed), 
    columns=column_order
)
for col in num_cols:
    X_val_preprocessed[col] = pd.to_numeric(X_val_preprocessed[col], errors='coerce')
print("X_val_preprocessed shape:", X_val_preprocessed.shape)
X_val_preprocessed.info()

All good now! Let's move forward

## Trainining the model

In [ ]:
# Convert Date to datetime
X_preprocessed['Date'] = pd.to_datetime(X_preprocessed['Date'])
X_val['Date'] = pd.to_datetime(X_val['Date'])

In [ ]:
X_preprocessed = X_preprocessed.drop(['Date','Place_ID'], axis=1)

In [ ]:
#training the model
reg = LinearRegression().fit(X_preprocessed, y_train)

In [ ]:
y_train_pred = reg.predict(X_preprocessed)
mse = mean_squared_error(y_train, y_train_pred)
print(mse)

In [ ]:
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
r2 = r2_score(y_train, y_train_pred)
print(r2)

Building the Pipeline for the Random Forest and the Optimization step via GridSearch

In [ ]:
# Building a Full Pipeline for a Random Forest Regressor
pipe_rf = Pipeline([
    ('rf', RandomForestRegressor(random_state=42, max_depth=10, n_jobs=-1))
])

# Perform 5-fold crossvalidation
y_train_pred_rf = cross_val_predict(pipe_rf, X_preprocessed, y_train, cv=5)

# Calculating the RSME and R²
mse_rf = mean_squared_error(y_train, y_train_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_train, y_train_pred_rf)

print('Cross validation scores:')
print('-------------------------')
print(f'RSME: {rmse_rf}')
print(f'R²: {r2_rf}')

My original parameter settings was:

*'rf__n_estimators': [50, 100, 200, 300, 500],      
*'rf__max_depth': [5, 10, 15, 20, 30, None],          
*'rf__min_samples_split': [2, 5, 10],          
*'rf__min_samples_leaf': [1, 2, 4],            
*'rf__max_features': ['sqrt', 'log2', None],  
*'rf__bootstrap': [True, False]

But the predicted time (calculaed by Claude) would have been by several hours...
I interrupted after running 80 minutes with the below section for GridSearch (comment out) and tried RandomizedSearch

In [ ]:
# Define parameter space for Random Forest
# Since we want to access the Random Forest step (called 'rf') in our pipeline,
# we add 'rf__' in front of the corresponding hyperparameters
from sklearn.model_selection import RandomizedSearchCV


param_rf = {
    'rf__n_estimators': [100, 200],       
    'rf__max_depth': [10, 20, None],           
    'rf__min_samples_split': [2, 10],          
    'rf__min_samples_leaf': [1, 4],            
    'rf__max_features': ['sqrt', None],  
    'rf__bootstrap': [True, False]                  
}

#grid_rf = GridSearchCV(pipe_rf, param_grid=param_rf, cv=5, scoring='neg_mean_squared_error',
                       #verbose=2, n_jobs=-1, return_train_score=True)

# Fitting the model
#grid_rf.fit(X_preprocessed, y_train)

random_rf = RandomizedSearchCV(pipe_rf, param_distributions=param_rf, n_iter=20, cv=5, 
                               scoring='neg_mean_squared_error', verbose=10, n_jobs=-1, random_state=42, return_train_score=True
)
random_rf.fit(X_preprocessed, y_train)

# Best parameters from RandomizedSearchCV
print("Best parameters found: ", random_rf.best_score_)
print("Best parameters found: ", random_rf.best_params_)

# Save best model (including fitted preprocessing steps) as best_model 
best_rf_model = random_rf.best_estimator_
best_rf_model